# How to execute shell scripts

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * Action `run` uses `bash` under linux and `batch` under windows
  * Other shell scripting actions calls respective shells
  

##  Action `run`

`run` is the most frequently used action in sos. In most cases, it is similar to action `bash` and uses `bash` to execute specified script. Under the hood, this action is quite different from `bash` because the run action does not have a default interpreter and would behave differently under different situations.

### Executing a list of commands

In the simplest case when one or more commands are specified, action `run` would assume it is a batch script under windows, and a bash script otherwise.

In [1]:
run:
    echo "A"

echo "A"


A


It is different from an `bash` action in that

* It will **print the commands that re executed before execute**
* It will **exit with error if any of the commands exits with non-zero code**

For example, whereas a `bash` action would print an error message but continue as follows

In [2]:
bash:
    echoo "This is wrong"
    echo "This is correct"

/var/folders/ys/gnzk0qbx5wbdgm531v82xxljv5yqy8/T/tmp1nykq5tz.sh: line 1: echoo: command not found


This is correct


The `run` action would exit with error

In [3]:
%env --expect-error

run:
    echoo "This is wrong"
    echo "This is correct"

echoo "This is wrong"


/var/folders/ys/gnzk0qbx5wbdgm531v82xxljv5yqy8/T/tmpiycbcidi.sh: line 1: echoo: command not found


ExecuteError: [0]: 
Failed to execute ``/bin/bash -ev .sos/scratch_0_0_fca38a45.sh``
exitcode=127, workdir=``/Users/bpeng1/sos/sos-docs/src/user_guide``
---------------------------------------------------------------------------

In another word,
```
run:
    command1
    command2
    command3
```
is equivalent to

```
bash:
    command1 && command2 && command3
```
under Linux/MacOS systems.

### Using shebang-specified interpreter

If the script starts with a shebang line, this action would execute the script directly. This allows you to execute any script in any language. For example, the following script executes a python script using action `run`

In [4]:
run:
    #!/usr/bin/env python
    print('This is python')

This is python


and the following example runs a complete sos script using command `sos-runner`

In [5]:
run:
    #!/usr/bin/env sos-runner
    [10]
    print(f"This is {step_name}")
    [20]
    print(f"This is {step_name}")

INFO: Running 10: !/usr/bin/env sos-runner


This is 10


INFO: 10 is completed.


INFO: Running 20: 


This is 20


INFO: 20 is completed.


INFO: Workflow default (ID=55b3a265b673438c) is executed successfully with 2 completed steps.


Note that action `run`would not analyze shebang line of a script if it is executed in a docker container (with option `docker-image`) and would always assumed to be `bash`.

## Action  `bash`

Action `bash(script)` accepts a shell script and execute it using `bash`. 

In [6]:
bash:
    for (( counter=10; counter>0; counter-- ))
    do
        echo -n "$counter "
    done
    printf "\n"

10 9 8 7 6 5 4 3 2 1 


## Action `sh`

Execute script with a `sh` interpreter. On most systems it is just an alias to `bash`. 

In [7]:
n = 10
sh: expand=True
    if [ {n} -lt 10 ];
    then
        echo "It is a one digit number"
    else
        echo "It is a two digit number"
    fi

It is a two digit number


## Action  `csh`

Action `csh` executes specified script with a `csh` interpreter

In [8]:
csh:
    set name1 = abc
    set name = cde
    set text = "File $name""1 is not $name1"
    echo $text

File cde1 is not abc


## Action  `tcsh`

Action `tcsh` executes specified script with a `tcsh` interpreter

In [9]:
tcsh:
    set testing = 10
    if ($testing == 10) echo "it worked"
    echo "This is the end of the test"

it worked


This is the end of the test


##  Action `zsh`

Action `zsh` executes script with a `zsh` interpreter

In [10]:
zsh:
    wordlist="one two three"
    wordarray=( $wordlist )

    for word in $wordarray; do
        echo "->$word<-"
    done

->one two three<-


## Further reading

* [SoS actions](sos_actions.html)
* [Script execution actions](script_actions.html)